In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install python-dotenv
!pip install PyPDF2
!pip install python-docx
!pip install faiss-cpu
!pip install streamlit
!pip install python-dotenv
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile RAG_Chatbot.py
from langchain.chains.question_answering import load_qa_chain
import openai
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import docx
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain import FAISS
import streamlit as st
import io
import os
from pyngrok import ngrok
def read_pdf(uploaded_file):
    text = ""
    with io.BytesIO(uploaded_file.getvalue()) as file_stream:
      pdf_reader = PdfReader(file_stream)
      for page in pdf_reader.pages:
        text += (page.extract_text() or "")
    return text
def read_docx(uploaded_file):
    text = ""
    doc = docx.Document(io.BytesIO(uploaded_file.getvalue()))
    text = '\n'.join(paragraph.text for paragraph in doc.paragraphs if paragraph.text)
    return text
def process_files(uploaded_files):
    combined_text = ""
    try:
      for uploaded_file in uploaded_files:
        if uploaded_file.name.endswith('.pdf'):
            combined_text += read_pdf(uploaded_file)
        elif uploaded_file.name.endswith('.docx'):
            combined_text += read_docx(uploaded_file)
    except:
      st.error("Please upload a PDF or docx files")
    return combined_text
def main():
    load_dotenv()
    st.set_page_config(page_title="Document Chatbot", page_icon=":robot:")
    robot_emoji = '\U0001F916'
    st.title(f"Docubot {robot_emoji}")
    uploaded_files = st.file_uploader("Upload your PDF or DOCX files", type=["pdf", "docx"], accept_multiple_files=True)

    if uploaded_files:
        train_documents = uploaded_files
        text = process_files(train_documents)
        char_text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len)
        chunks = char_text_splitter.split_text(text)
        embeddings = OpenAIEmbeddings()
        docsearch = FAISS.from_texts(chunks, embeddings)
        llm = OpenAI(temperature=0.7)
        chain = load_qa_chain(llm, chain_type="stuff")
        query = st.text_input("Type your question here and press enter")
        if query:
          docs = docsearch.similarity_search(query)
          with st.spinner("Loading the answer..."):
            result = chain.run(input_documents=docs, question=query)
            st.write(result)
if __name__ == "__main__":
    main()

Overwriting RAG_Chatbot.py


In [ ]:
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token('replace_ngrok_token')
public_url = ngrok.connect(8501)
print('Streamlit is running at:', public_url)
!streamlit run --server.port 8501 RAG_Chatbot.py >/dev/null &

Streamlit is running at: NgrokTunnel: "https://fa01-34-29-12-175.ngrok-free.app" -> "http://localhost:8501"
/usr/local/lib/python3.10/dist-packages/langchain/_api/module_import.py:87: LangChainDeprecationWarning: Importing GuardrailsOutputParser from langchain.output_parsers is deprecated. Please replace the import with the following:
from langchain_community.output_parsers.rail_parser import GuardrailsOutputParser
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/__init__.py:35: LangChainDeprecationWarning: Importing vector stores from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.vectorstores import FAISS`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py:548: LangChainDeprecationWarning: Importing LLMs from langchain 

In [ ]:
ngrok.kill()